https://qiita.com/ytaki0801/items/fc785de02eb45cde94fa

In [1]:
fib21 = 
[[["lambda", ["u"], ["u", "u"]],
        ["lambda", ["u"],
            ["lambda", ["n", "a", "b"],
                ["if", ["=", "n", 0], "a",
                    [["u", "u"], ["-", "n", 1], "b", ["+", "a", "b"]]]]]],
    21, 0, 1]

g = Dict("="=>"==","+"=>"+","-"=>"-")
function ev(s, e)
    if isa(s, String) return merge(g, e)[s]
    elseif isa(s, Int) return s
    elseif s[1] == "if" return ev(s[2], e) ? ev(s[3], e) : ev(s[4], e)
    elseif s[1] == "lambda" return vcat(s, [e])
    else
        f = ev(s[1], e); a = map(x -> ev(x, e), s[2:end])
        if isa(f, String) return eval(Expr(:call, Symbol(f), a...))
        else return ev(f[3], merge(f[4], Dict(k for k = zip(f[2], a))))
        end
    end
end

@show ev(fib21, Dict());

ev(fib21, Dict()) = 10946


In [2]:
fib21 = 
[[["lambda", ["u"], ["u", "u"]],
        ["lambda", ["u"],
            ["lambda", ["n", "a", "b"],
                ["if", ["=", "n", 0], "a",
                    [["u", "u"], ["-", "n", 1], "b", ["+", "a", "b"]]]]]],
    21, 0, 1]

g = Dict("=" => "==", "+" => "+", "-" => "-")
function ev(s, e)
    s isa String     && return merge(g, e)[s]
    s isa Int        && return s
    s[1] == "if"     && return ev(s[2], e) ? ev(s[3], e) : ev(s[4], e)
    s[1] == "lambda" && return vcat(s, [e])
    f, a = ev(s[1], e), ev.(s[2:end], Ref(e))
    f isa String     && return eval(Expr(:call, Symbol(f), a...))
    return ev(f[3], merge(f[4], Dict(zip(f[2], a))))
end

@show ev(fib21, Dict());

ev(fib21, Dict()) = 10946


In [3]:
fib21 = 
[[["lambda", ["u"], ["u", "u"]],
        ["lambda", ["u"],
            ["lambda", ["n", "a", "b"],
                ["if", ["=", "n", 0], "a",
                    [["u", "u"], ["-", "n", 1], "b", ["+", "a", "b"]]]]]],
    21, 0, 1]

g = Dict("=" => ==, "+" => +, "-" => -)
function ev(s, e)
    s isa String     && return merge(g, e)[s]
    s isa Int        && return s
    s[1] == "if"     && return ev(s[2], e) ? ev(s[3], e) : ev(s[4], e)
    s[1] == "lambda" && return vcat(s, [e])
    f, a = ev(s[1], e), ev.(s[2:end], Ref(e))
    f isa Function   && return f(a...)
    return ev(f[3], merge(f[4], Dict(zip(f[2], a))))
end

@show ev(fib21, Dict());

ev(fib21, Dict()) = 10946


In [4]:
fib21 = 
[[[:lambda, [:u], [:u, :u]],
        [:lambda, [:u],
            [:lambda, [:n, :a, :b],
                [:if, [:(=), :n, 0], :a,
                    [[:u, :u], [:-, :n, 1], :b, [:+, :a, :b]]]]]],
    21, 0, 1]

g = Dict(:(=) => ==, :+ => +, :- => -)
function ev(s, e)
    s isa Symbol     && return merge(g, e)[s]
    s isa Int        && return s
    s[1] === :if     && return ev(s[2], e) ? ev(s[3], e) : ev(s[4], e)
    s[1] === :lambda && return vcat(s, [e])
    f, a = ev(s[1], e), ev.(s[2:end], Ref(e))
    f isa Function   && return f(a...)
    return ev(f[3], merge(f[4], Dict(zip(f[2], a))))
end

@show ev(fib21, Dict());

ev(fib21, Dict()) = 10946


In [5]:
fib21 = 
[[["lambda", ["u"], ["u", "u"]],
        ["lambda", ["u"],
            ["lambda", ["n", "a", "b"],
                ["if", ["=", "n", 0], "a",
                    [["u", "u"], ["-", "n", 1], "b", ["+", "a", "b"]]]]]],
    21, 0, 1]

g = Dict("=" => :(==), "+" => :+, "-" => :-)
function ev(s)
    s isa String     && return get(g, s, Symbol(s))
    s isa Int        && return s
    s[1] == "if"     && return Expr(:if, ev(s[2]), ev(s[3]), ev(s[4]))
    s[1] == "lambda" && return Expr(:(->), Expr(:tuple, Symbol.(s[2])...), ev(s[3]))
    Expr(:call, ev.(s)...)
end
macro ev(x) ev(eval(x)) end

@show @ev fib21;

#= In[5]:19 =# @ev(fib21) = 10946


In [6]:
ev(fib21)

:(((((u,)->u(u)))(((u,)->((n, a, b)->if n == 0
                  a
              else
                  (u(u))(n - 1, b, a + b)
              end))))(21, 0, 1))

In [7]:
Fib = @ev [["lambda", ["u"], ["u", "u"]],
    ["lambda", ["u"],
        ["lambda", ["n", "a", "b"],
            ["if", ["=", "n", 0], "a",
                [["u", "u"], ["-", "n", 1], "b", ["+", "a", "b"]]]]]]

@show Fib(21, 0, 1)
@show [Fib(n, 0, 1) for n in 1:10];

Fib(21, 0, 1) = 10946
[Fib(n, 0, 1) for n = 1:10] = [1, 1, 2, 3, 5, 8, 13, 21, 34, 55]


In [8]:
@code_native debuginfo=:none Fib(21, 0, 1)

	.text
	movq	%rdx, %rax
	testq	%rcx, %rcx
	je	L47
	pushq	%rbp
	movq	%rsp, %rbp
	subq	$32, %rsp
	decq	%rcx
	addq	%r8, %rax
	movabsq	$.text, %r9
	movq	%r8, %rdx
	movq	%rax, %r8
	callq	*%r9
	addq	$32, %rsp
	popq	%rbp
	retq
L47:
	retq


In [9]:
fib21 = 
[[[:lambda, [:u], [:u, :u]],
        [:lambda, [:u],
            [:lambda, [:n, :a, :b],
                [:if, [:(==), :n, 0], :a,
                    [[:u, :u], [:-, :n, 1], :b, [:+, :a, :b]]]]]],
    21, 0, 1]

g = Dict(:(=) => :(==), :+ => :+, :- => :-)
function ev(s)
    s isa Symbol     && return get(g, s, s)
    s isa Int        && return s
    s[1] === :if     && return Expr(:if, ev(s[2]), ev(s[3]), ev(s[4]))
    s[1] === :lambda && return Expr(:(->), Expr(:tuple, s[2]...), ev(s[3]))
    Expr(:call, ev.(s)...)
end
macro ev(x) ev(eval(x)) end

@show @ev fib21;

#= In[9]:19 =# @ev(fib21) = 10946


In [10]:
ev(fib21)

:(((((u,)->u(u)))(((u,)->((n, a, b)->if n == 0
                  a
              else
                  (u(u))(n - 1, b, a + b)
              end))))(21, 0, 1))

In [11]:
Fib = @ev [[:lambda, [:u], [:u, :u]],
        [:lambda, [:u],
            [:lambda, [:n, :a, :b],
                [:if, [:(==), :n, 0], :a,
                    [[:u, :u], [:-, :n, 1], :b, [:+, :a, :b]]]]]]

@show Fib(21, 0, 1)
@show [Fib(n, 0, 1) for n in 1:10];

Fib(21, 0, 1) = 10946
[Fib(n, 0, 1) for n = 1:10] = [1, 1, 2, 3, 5, 8, 13, 21, 34, 55]


In [12]:
@code_native debuginfo=:none Fib(21, 0, 1)

	.text
	movq	%rdx, %rax
	testq	%rcx, %rcx
	je	L47
	pushq	%rbp
	movq	%rsp, %rbp
	subq	$32, %rsp
	decq	%rcx
	addq	%r8, %rax
	movabsq	$.text, %r9
	movq	%r8, %rdx
	movq	%rax, %r8
	callq	*%r9
	addq	$32, %rsp
	popq	%rbp
	retq
L47:
	retq
